# Show Notebook for Usage of sort_neigh.py

This jupyter notebook shows the core features of sort_neigh.py.

sort_neigh.py consists of a classifier (`sort_neigh.NeighbourClassifier`) and a surrounding class (`sort_neigh.NeighbourSort`) that handles reading the given nanoparticle and writing the results to according folders.
The key features can also be used in command line, some examples of how to do this are given below.

## Imports

Importing sort_neigh.py and time for timing the function calls.

In [ ]:
import sys
sys.path.append("../py_src")

import sort_neigh
import time

## Target Definition

Set the original nanoparticle over a given amount of timesteps in `nano_path`.
An output file for where to save the resulting counts per category for each timestep can be supplied in `save_txt_path`.
Initial configurations for the neighbour sorting algorithm are made here.

In [ ]:
nano_path = "../test_data/cunano_show.lammpstrj"
save_txt_path = "../test_data/show_analysis.txt"
sorter = sort_neigh.NeighbourSort(rcut=3.1, nmax=4, lmax=3, sigma=0.5, gamma_kernel=0.05)

## Running the sorting algorithm

`NeighbourSort.init_folder_structure` creates one subdirectory in `out_dir` for each timestep in `nano_path`.
Each subdirectory contains the nanoparticle at the given timestep as a .lammpstrj file.

WARNING: `n_atoms_in_part` (number of atoms in nanoparticle) and `timesteps` (number of timesteps in `nano_path`) need to be set correctly for `NeighbourSort` to work. 

Running `NeighbourSort.create_local_structure` calculates the number of particles in a given configuration and saves it as a numpy array.
Depending on `mode` the atoms are either pre-filtered by their number of neighbours ("pre_group") or compared to the full kernel ("class_all").
If the `create_subfolders` flag is set, each timestep directory is filled with directories containing the nearest neighbours structures of the atom.

`NeighbourSort.sort_save_cat` is used to sort and save a category counter for the current configuration of variables in `NeighbourSort`.

The following command line arguments perform the same functionalities when run from the surface_class directory:

"`sort_neigh_terminal.py sort_cat test_data/cunano_show.lammpstrj -o test_data/sort_neigh_results/ -O test_data/show_analysis.txt`"

"`sort_neigh_terminal.py sort_cat test_data/cunano_show.lammpstrj -o test_data/sort_neigh_results/ -O test_data/show_analysis.txt --r_cut 0.1 --create_subfolders 0 --classify_mode class_all`"

In [ ]:
start = time.time()
sorter.init_folder_structure(
    nano_path, n_atoms_in_part=1400, timesteps=100, out_dir="../test_data/sort_neigh_results/"
)

cat_counter = sorter.create_local_structure(
    last_n=14, create_subfolders=True,
    mode="pre_group"
)
stop = time.time()
print("Sorting took %u seconds."%(stop-start))

file_name = sorter.sort_save_cat(
    file_name=save_txt_path, cat_counter=cat_counter
)

## Plotting Results

`NeighbourSort` contains a handy function for plotting pre-existing results called `NeighbourSort.plot_dist`.
Here, a previously saved category counter is loaded via `NeighbourSort.load_sort_cat` and plotted.

The following command line arguments perform the same functionalities when run from the surface_class directory:

"`sort_neigh_terminal.py plot_result test_data/show_analysis.txt`"

In [ ]:
sorted_cat_counter, timesteps, sorted_classes = sorter.load_sort_cat(file_name=save_txt_path)
sorter.plot_dist(sorted_classes, sorted_cat_counter)